In [ ]:
"""
PLAN (BabI task 1)

given input:
- fact 1   | John travelled to the hallway.
- fact 2   | Mary journeyed to the bathroom.
- question | Where is John?

output:
- answer   | hallway
"""

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

pipeline = Pipeline([
    ('bow', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
#    ('clf', ExtraTreesClassifier()),
    ('clf', RandomForestClassifier()),
])

parameters = {
    'clf__n_estimators': [10, 50, 100],
    'clf__criterion': ['gini', 'entropy'],
    'clf__max_depth': [1, 2, 3, 5, 8, 13],
    'clf__max_features': [1, 2, 3, 5, 8, 13],
}

import pandas as pd
from parse_dataset import parse_data

df_train = parse_data('/Users/enrico.t/code/ml-reference-implementations/' + 
                      'data/babI/tasks_1-20_v1-2/en/qa1_single-supporting-fact_train.txt')

X_train = df_train['fact1'] + ' ' + df_train['fact2'] + ' ' + df_train['question']
y_train = df_train['answer']

df_test = parse_data('/Users/enrico.t/code/ml-reference-implementations/' + 
                     'data/babI/tasks_1-20_v1-2/en/qa1_single-supporting-fact_test.txt')

X_test = df_test['fact1'] + ' ' + df_test['fact2'] + ' ' + df_test['question']
y_test = df_test['answer']

model = GridSearchCV(
    pipeline, 
    parameters, 
    cv=5,
    n_jobs=-1, 
    verbose=1
)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

print model.best_params_

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   47.1s


In [ ]:
from pandas_ml import ConfusionMatrix

cm = ConfusionMatrix(y_test, y_predict)
cm.print_stats()
print cm.plot()